# Price-taker with uncertainty analysis notebook

In [3]:
import numpy as np
import pandas as pd
from numbers import Real
from pathlib import Path
from functools import partial
# from Backcaster_of_price_taker import PricetakerBackcaster
from Backcaster_of_price_taker import PriceBackcaster
from dispatches.case_studies.renewables_case.load_parameters import *

In [8]:
lmps_df = pd.read_parquet(Path("data", "303_LMPs_15_reserve_500_shortfall.parquet"))
lmps = lmps_df['LMP'].values
lmps[lmps>500] = 500

In [9]:
def check_two_signals(signal_1, signal_2):
    return np.allclose(signal_1, signal_2)

In [11]:
# define the initial values
signal = lmps
horizon = 24
future_horizon = 48
scenario = 3

pb = PriceBackcaster(signal, scenario=scenario, pointer=0, horizon=horizon, future_horizon=future_horizon)

real_signal = lmps[0:24]
fetched_signal = pb.generate_stage_1_price_signals()
check_two_signals(real_signal, fetched_signal)

True

In [16]:
real_signal_1 = lmps[-48:]
fetched_signal_1 = pb.generate_stage_2_price_signals()[2]
check_two_signals(real_signal_1, fetched_signal_1)

True

In [18]:
real_signal_2 = lmps[-72:-24]
fetched_signal_2 = pb.generate_stage_2_price_signals()[1]
check_two_signals(real_signal_2, fetched_signal_2)

True

In [20]:
real_signal_3 = lmps[-96:-48]
fetched_signal_3 = pb.generate_stage_2_price_signals()[0]
check_two_signals(real_signal_3, fetched_signal_3)

True

In [22]:
pb.pointer = 10

real_signal = lmps[10*24: 10*24+24]
fetched_signal = pb.generate_stage_1_price_signals()
check_two_signals(real_signal, fetched_signal)

True

In [24]:
real_signal_1 = lmps[(pb.pointer-2)*24:pb.pointer*24]
fetched_signal_1 = pb.generate_stage_2_price_signals()[2]
check_two_signals(real_signal_1, fetched_signal_1)

True

In [25]:
real_signal_2 = lmps[(pb.pointer-3)*24:(pb.pointer-1)*24]
fetched_signal_2 = pb.generate_stage_2_price_signals()[1]
check_two_signals(real_signal_2, fetched_signal_2)

True

In [26]:
real_signal_3 = lmps[(pb.pointer-4)*24:(pb.pointer-2)*24]
fetched_signal_3 = pb.generate_stage_2_price_signals()[0]
check_two_signals(real_signal_3, fetched_signal_3)

True

## Inspect the test results

In [112]:
import json
path = "test_wb_new_uncertainty_scenario3_24_48.json"
with open(path, 'rb') as f:
    res_dict = json.load(f)

res_dict['0']

{'stage_1_start_soc': 0,
 'stage_1_end_soc': 88895.61261435537,
 'stage_2_start_soc': [88895.61261435537,
  88895.61261435537,
  88895.61261435537],
 'stage_2_end_soc': [0.0, 0.0, 0.0],
 'stage_1_end_energy_throughput': 351519.28554662404,
 'solver_stat': 'optimal',
 'stage_1_elec_in': 46124.19192320253,
 'stage_1_elec_out': 0.015400942370527929,
 'stage_1_init_et': 0.16762722639093575,
 'stage_1_step_0_et': 23062.271289298846,
 'revenue': 888235.2366424422}

In [113]:
for key in res_dict.keys():
    print(res_dict[key]['solver_stat'])

optimal
optimal
optimal
optimal
optimal
optimal
optimal
optimal
optimal
optimal
optimal
optimal
optimal
optimal
optimal
optimal
optimal
optimal
optimal
optimal


In [116]:
rev=0
for key in res_dict.keys():
    rev += res_dict[key]["revenue"]
    print(res_dict[key]['stage_1_start_soc'], res_dict[key]['stage_1_end_soc'])
    print(res_dict[key]['stage_2_start_soc'], res_dict[key]['stage_2_end_soc'])
    print("--------------------------------------------------------------------")
print(rev)

0 88895.61261435537
[88895.61261435537, 88895.61261435537, 88895.61261435537] [0.0, 0.0, 0.0]
--------------------------------------------------------------------
88895.61261435537 169369.87847814368
[169369.87847814368, 169369.87847814368, 169369.87847814368] [88895.61261435537, 88895.61261435537, 88895.61261435537]
--------------------------------------------------------------------
169369.87847814368 169388.5548000933
[169388.5548000933, 169388.5548000933, 169388.5548000933] [169369.87847814368, 169369.87847814368, 169369.87847814368]
--------------------------------------------------------------------
169388.5548000933 151574.70063821285
[151574.70063821285, 151574.70063821285, 151574.70063821285] [169388.55480009314, 169388.55480009344, 169388.5548000933]
--------------------------------------------------------------------
151574.70063821285 169399.06153708944
[169399.06153708944, 169399.06153708944, 169399.06153708944] [151574.70063821285, 151574.70063821285, 151574.70063821285]


In [58]:
for i in range(10):
    print(f"day_{i}  ", lmps[i*24:(i+1)*24])
    print("----------------------------------")

day_0   [ 0.        0.        0.        0.        0.        0.        0.
 -0.        0.        0.        0.        0.       18.861019 19.034775
  0.        0.       19.430296 26.486943 23.069973 22.732362 21.474047
 20.419098 22.492854 21.474047]
----------------------------------
day_1   [21.287879 72.185841 72.492854 72.96816  71.843654 20.419098 21.287879
 73.069973  0.        0.        0.        0.        0.        0.
  0.        0.        0.        0.        0.        0.       -0.
 -0.        0.       -0.      ]
----------------------------------
day_2   [ 0.        0.        0.        0.       19.68943  -0.        0.
  0.        0.        0.        0.        0.        0.        0.
  0.        0.        0.        0.       18.861019 19.68943  22.576747
 23.069973 23.128644 22.576747]
----------------------------------
day_3   [23.128644 23.128644 23.069973 23.069973 22.96816  23.128644 24.617414
 76.324557 22.492854 19.034775  0.        0.        0.        0.
  0.        0.        